In [1]:
import json
import pandas as pd

from glob import glob
from collections import defaultdict

In [2]:
traces = glob("./logs/**/*.json", recursive=True)
total_durs = {}

for trace in traces:
    with open(trace) as f:
        json_dict = json.load(f)

    total_dur = {}

    for event in json_dict["traceEvents"]:
        if not "dur" in event:
            continue

        cat = event["cat"]
        dur = event["dur"]
        name = event["name"]

        if not dur:
            continue

        if not cat in total_dur:
            total_dur[cat] = defaultdict(int)

        total_dur[cat][name] += dur

    rows = []
    for cat, sub_dict in total_dur.items():
        for name, dur in sub_dict.items():
            rows.append({
                "cat": cat,
                "name": name,
                "dur": dur,
            })

    total_durs[trace.split("/")[-2]] = pd.DataFrame(rows)

In [3]:
targets = {"aten::": "ManipulateTensor", "autograd::": "CalculateGradient", "Backward": "Backward"}

for key, val in total_durs.items():
    print(key)
    for target, alt_name in targets.items():
        len_ori = len(val)
        df_temp = val[val["name"].str.contains(target)]
        val.loc[len_ori] = ({"cat": "cpu_op", "name": alt_name, "dur": df_temp["dur"].sum()})
        val.drop(df_temp.index, inplace=True)
        val.reset_index(drop=True, inplace=True)
        print(f"{alt_name} (length: {len_ori} --> {len(val)})")
    print()

np8-bs8
ManipulateTensor (length: 217 --> 138)
CalculateGradient (length: 138 --> 113)
Backward (length: 113 --> 88)

np8-bs16
ManipulateTensor (length: 218 --> 139)
CalculateGradient (length: 139 --> 114)
Backward (length: 114 --> 89)



In [5]:
pd.set_option("display.max_rows", 10)  # Display all rows
pd.set_option("display.max_columns", None)  # Display all columns
pd.set_option("display.max_colwidth", 60)

for idx, (key, val) in enumerate(total_durs.items()):
    val = val.sort_values(by=["cat", "dur"], ascending=[True, False])
    val.reset_index(drop=True, inplace=True)
    trace = traces[idx]
    val.to_csv(f"{trace[:trace.rfind('/')]}/trace.csv")

    allreduce_c = val[(val["cat"] == "cpu_op") & (val["name"] == "c10d::allreduce_")]["dur"].item()
    cpu_op = val[val["cat"] == "cpu_op"]["dur"].sum()
    allreduce_k = val[(val["cat"] == "kernel") & (val["name"].str.contains("AllReduce"))]["dur"].item()
    kernel = val[val["cat"] == "kernel"]["dur"].sum()

    display(val[val["cat"] == "cpu_op"])
    print(f"[{key}] percent of allreduce in cpu operation: {round(allreduce_c / cpu_op * 100, 4)}%)")
    display(val[val["cat"] == "kernel"])
    print(f"[{key}] percent of allreduce in kernel: {round(allreduce_k / kernel * 100, 4)}%)\n")

,cat,name,dur
1,cpu_op,ManipulateTensor,645995
2,cpu_op,CalculateGradient,218030
3,cpu_op,Backward,124125
4,cpu_op,c10d::allreduce_,17623
5,cpu_op,record_param_comms,16324
6,cpu_op,GeLUFunction,10688
7,cpu_op,torch.distributed.ddp.reducer::copy_bucket_to_grad,8883
8,cpu_op,detach_,6


[np8-bs8] percent of allreduce in cpu operation: 1.6918%)


,cat,name,dur
26,kernel,"ncclKernel_AllReduce_RING_LL_Sum_float(ncclDevComm*, uns...",185576
27,kernel,"void at::native::vectorized_elementwise_kernel<4, at::na...",38607
28,kernel,void at::native::unrolled_elementwise_kernel<at::native:...,31647
29,kernel,void at::native::unrolled_elementwise_kernel<at::native:...,27397
30,kernel,ampere_fp16_s16816gemm_fp16_128x128_ldg8_f2f_stages_32x5_nt,26411
...,...,...,...
77,kernel,"void at::native::vectorized_elementwise_kernel<4, at::na...",8
78,kernel,"at::native::amp_update_scale_cuda_kernel(float*, int*, f...",8
79,kernel,void (anonymous namespace)::elementwise_kernel_with_inde...,6
80,kernel,void (anonymous namespace)::elementwise_kernel_with_inde...,6


[np8-bs8] percent of allreduce in kernel: 34.8915%)



,cat,name,dur
1,cpu_op,ManipulateTensor,854112
2,cpu_op,CalculateGradient,262026
3,cpu_op,Backward,155811
4,cpu_op,torch.distributed.ddp.reducer::copy_bucket_to_grad,28134
5,cpu_op,c10d::allreduce_,18492
6,cpu_op,record_param_comms,17045
7,cpu_op,GeLUFunction,10223
8,cpu_op,detach_,5


[np8-bs16] percent of allreduce in cpu operation: 1.374%)


,cat,name,dur
27,kernel,"ncclKernel_AllReduce_RING_LL_Sum_float(ncclDevComm*, uns...",174951
28,kernel,ampere_fp16_s16816gemm_fp16_128x128_ldg8_f2f_stages_32x5_nt,54822
29,kernel,"void at::native::vectorized_elementwise_kernel<4, at::na...",39017
30,kernel,void at::native::unrolled_elementwise_kernel<at::native:...,36486
31,kernel,"void at::native::vectorized_elementwise_kernel<4, at::na...",34326
...,...,...,...
78,kernel,"at::native::amp_update_scale_cuda_kernel(float*, int*, f...",8
79,kernel,"void at::native::vectorized_elementwise_kernel<4, at::na...",7
80,kernel,void (anonymous namespace)::elementwise_kernel_with_inde...,6
81,kernel,void (anonymous namespace)::elementwise_kernel_with_inde...,6


[np8-bs16] percent of allreduce in kernel: 25.9832%)

